In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install tensorflow-gpu    #uncomment it firstly, if the code throws an error   
# !pip install --upgrade pip     #uncomment it secondly, if the code throws an error

from tensorflow.python.client import device_lib
print([x.name for x in device_lib.list_local_devices()])

In [ ]:
#Imports


import json
import numpy as np
import copy
from random import shuffle, seed
import sys
import os.path
import argparse
import glob
import scipy.io
import pdb
import string
import h5py
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import gensim
import re
import cv2
import tensorflow as tf
import keras
import pandas as pd
import matplotlib.pyplot as plt
dropout_rate = 0.4
import warnings
warnings.filterwarnings('ignore')
import os
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import keras.activations
import keras.backend as kbe
from keras.callbacks import EarlyStopping
import tensorflow.keras.layers as layers
from keras.layers import Activation, Add, Concatenate, Conv1D, Dense, Dropout, Embedding, Softmax
from keras.layers import Input, GlobalMaxPooling1D, Lambda, Multiply, RepeatVector, Reshape
from keras.layers import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
import pickle
from pprint import pprint

In [ ]:
!wget https://figshare.com/ndownloader/files/10798046 -O GoogleNews-vectors-negative300.bin  #Use it to download on kaggle storage


model_path = '/kaggle/working/GoogleNews-vectors-negative300.bin'            # Path where the model is stored
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)        #word_to_vector model initiated

In [ ]:
!mkdir /kaggle/working/QA                 #Add it to stor .h5 files
!mkdir /kaggle/working/VQA_Model          #Add it to store the final model

In [ ]:
def extract_feat(doc):
  '''
    Input: A sentence
    Output : Word Embedding of the sentence

    Note : In order to maintain uniformity in the word embedding of the question, I have used padding (which would be shown later) with a maximum length of 21 (since this
    is the longest length of the question in VQA_RAD Dataset Public.json)
  '''
  feat = []
  for word in doc:
      try:
          feat.append(model_w2v[word])
      except:
          pass
  return feat

def tokenize(sentence):

    ''' Perform Tokenization '''
    return [i for i in re.split(r"([-.\"',:? !$#@~()*&\^%;/\\+<>\n=])", sentence) if i!='' and i!=' ' and i!='\n'];

def prepro_question(imgs, method):
    # preprocess all the question
    print('example processed tokens:')

    '''
    Input: The question from VQA_RAD Dataset Public.json
    Performs tokenization and lowering of the question
    Output: Embedded version of the question

    Note that: We have still not padded the questions, just for information each of the word will be a 300 Dimensional Vector,
    hence, the word vector (which will be obtained after padding) will be a (21,300) Dimensional
    '''
    for i,img in enumerate(imgs):
        s = img['question'].lower()
        if method == 'nltk':
            txt = word_tokenize(str(s).lower())
        else:
            txt = tokenize(s)
        img['processed_tokens'] = txt
        if i < 10: print(txt)
        if i % 1000 == 0:
            sys.stdout.write("processing %d/%d (%.2f%% done)   \r" %  (i, len(img), i*100.0/len(imgs)) )
            sys.stdout.flush()
    return imgs

def get_top_answers(imgs, num_ans):

    """
    Print the questions and returns the number of time one answer is repeated
    """
    counts = {}
    for img in imgs:
        try:
            ans = img['answer'].lower()  # If the string is a number, it would result into error
        except :
            ans = str(img['answer'])
        counts[ans] = counts.get(ans, 0) + 1

    cw = sorted([(count,w) for w,count in counts.items()], reverse=True)
    print('top answer and their counts:')
    print('\n'.join(map(str,cw[:20])))

    vocab = []
    for i in range(min(num_ans,len(cw))):
        vocab.append(cw[i][1])

    return vocab[:num_ans]


def filter_question(imgs, atoi):

  ''' Not of much use, I had used it for some other purpose, but did not use it later '''
  new_imgs = []
  for i, img in enumerate(imgs):
          new_imgs.append(img)

  print('question number reduce from %d to %d '%(len(imgs), len(new_imgs)))
  return new_imgs

manualMap = { 'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three':
                  '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7',
                  'eight': '8', 'nine': '9', 'ten': '10' }

imgs = json.load(open('/kaggle/input/vqa-rad-visual-question-answering-radiology/VQA_RAD Dataset Public.json' , 'r'))      
num_ans = 1000
top_ans = get_top_answers(imgs, num_ans)
atoi = {w:i for i,w in enumerate(top_ans)}                   # Word : Count
itoa = {i:w for i,w in enumerate(top_ans)}                   # Count : Word
feat_dim = 300                                               # 300 Dimensional Vector
imgs_data = json.load(open('/kaggle/input/vqa-rad-visual-question-answering-radiology/VQA_RAD Dataset Public.json' , 'r'))  # VQA_RAD Dataset Public.json
num_ans = 10    # Even 1 should work fine, but I had taken reference from COCO dataset, and hence, 10 (10 represents the top 10 answers to a picture)
method = 'nltk'
max_length = 21                    # Max Length of the question
dir_path = "/kaggle/working/QA"    # The path where we will be storing .h5 file
N = len(imgs_data)

image_path = '/kaggle/input/vqa-rad-visual-question-answering-radiology/VQA_RAD Image Folder'

def save_data():


        for i,img in enumerate(imgs_data):
           #print('X' , img['ques_id'])
            img_path = image_path+img['image_name']

            s = img['question']
            print(i,s)   # Print the number and the question
            if method == 'nltk':
                try:
                    txt = word_tokenize(str(s).lower())
                except :
                    txt = str(s)
            else:
                    txt = tokenize(s)

            img['processed_tokens'] = txt
            question_id = img['qid']
            feat = np.array(extract_feat(img['processed_tokens']))
            label_arrays = np.zeros((1, max_length, feat_dim), dtype='float32')
            label_length = min(max_length, len(feat)) # record the length of this sequence
            label_arrays[0, :label_length, :] = feat
            try:
                ans_arrays = atoi[img['answer'].lower()]
            except :
                ans_arrays = atoi[str(img['answer'])]

            f = h5py.File(os.path.join( dir_path , str(question_id) + '.h5'), "w")
            f.create_dataset("ques", dtype='float32', data=label_arrays)
            f.create_dataset("answers", dtype='uint32', data=ans_arrays)
            f.close()
        return

data = save_data()

In [ ]:
def image_layer(input_shape):
    '''
    Input : Shape of the image
    Output : VGG16 Preprocessing model
    '''
    base_model = tf.keras.applications.VGG16(input_shape=input_shape, include_top=False,weights='imagenet')
    base_model.trainable = False      # Do not train it
    x = base_model.layers[-2].output  # Shape would be (28*28*512)
    x = tf.reshape(x , [-1,x.shape[2]*x.shape[1] , x.shape[3]]) # Shape would be (1,784,512)
    x = tf.keras.layers.Dense(1024)
    return x

def vgg_preprocessing(model,image):
  ''' Takes a tensor as an input, and returns a pre processed version of the image'''
  return model(image)

def load_data():

        '''
        Input: Nothing

        Output: Returns list containing the following four elements in a tuple
        (preprocessed version of the image,embedded question,embedded answer, question id)
        '''

        images = []
        questions = []
        answers = []
        ids = []

        data = imgs_data   # VQA_RAD Dataset Public.json
        model = image_layer(input_shape = (448,448,3)) # Making VGG16 Model
        for i,img in enumerate(data):

            img_path = img['image_name']  # Image Name
            question_id = img['qid']      # Question id

            #label_arrays = np.zeros((1, max_length, feat_dim), dtype='float32') # Somethings are taken directly from

            with h5py.File(os.path.join(dir_path,str(question_id) + '.h5'),'r') as hf:
                question = hf['ques'][:]           # Embedded question
                answer = hf['answers'][()]               # Embedded answer


            image = cv2.imread(os.path.join('/kaggle/input/vqa-rad-visual-question-answering-radiology/VQA_RAD Image Folder',img_path) , cv2.IMREAD_COLOR) # Reading the image
            image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
            image = cv2.resize(image , (448,448)) # Reshape

            '''
            Small Note : I had not applied the VGG Preprocessing here, because I wanted to use the original shape of 448, and then, I would
            attach the VGG Model, while making the model, I think this makes sense :)
            '''
            #image = vgg_preprocessing(model,image)
            # Apply VGG16 Preprocessings

            images.append(image)
            questions.append(np.array(question))
            answers.append(np.array(answer))
            ids.append(question_id)
            if i%100==0:
              print("Processed =>",i,' which is',round(100*i/len(data),2),'%')

        questions = np.reshape(np.array(questions) , [-1,max_length,feat_dim])
        return (np.array(images) , questions ,np.array(answers) , np.array(ids))

imgs = json.load(open('/kaggle/input/vqa-rad-visual-question-answering-radiology/VQA_RAD Dataset Public.json' , 'r'))
num_ans = 1000
top_ans = get_top_answers(imgs, num_ans )
atoi = {w:i for i,w in enumerate(top_ans)}
itoa = {i:w for i,w in enumerate(top_ans)}
feat_dim = 300
imgs_data = json.load(open('/kaggle/input/vqa-rad-visual-question-answering-radiology/VQA_RAD Dataset Public.json' , 'r'))
num_ans = 10
method = 'nltk'
max_length = 21
dir_path = "/kaggle/working/QA"
N = len(imgs_data)

In [ ]:
''' These parameters are for some of the previous attempts, so ignore it, the main part is : datagen = load_data(), and I don't want to remove all of these,
because indeed there can be some ideas coming out from these lines of code'''

embed_size = 300
q_len = 21
height = 224
width = 224
lstm_units = 256
attention_dim = 512
num_output = 1000
max_questions = 3064

batch_size = 32
lr = 0.001
articles = ['a', 'an', 'the']
manualMap = { 'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three':
                  '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7',
                  'eight': '8', 'nine': '9', 'ten': '10' }


datagen = load_data()   # Load the Data

In [ ]:
images,questions,answers,ids =datagen[0],datagen[1],datagen[2],datagen[3]
print("Images have a size of:",images.shape)
print("Questions have a size of:",questions.shape)
print("Answers have a size of:",answers.shape)
print("Ids have a size of:",ids.shape)
dir_path = r'/kaggle/working/QA' # The directory where the .h5 file for each entry is saved
m = 100
for i in range(images.shape[0]):
    ans_array = answers[i]
    image_array = images[i]
    quest_array = questions[i]
    question_id = ids[i]
    f = h5py.File(os.path.join( dir_path , str(question_id) + '.h5'), "w") # Loading the 'h5 file
    f.create_dataset("ques", dtype='float32', data=quest_array) # Question Embedding
    f.create_dataset("image_vector", dtype='float32', data=image_array) # Image Embedding (Not preprocessed)
    f.create_dataset("answers", dtype='uint32', data=ans_array)      # Answers in embedded form
    f.close()
    if i%m ==0:
        print("Processed =>", i,' total percentage =>', round(100*i/images.shape[0],2),' %')
print("Your processing has been done")

In [ ]:
dir = r'/kaggle/working/QA/' # Containing .h5 file
images = []
ans = []
ques = []
count = 0
content = os.listdir(dir)
length = len(content)
for i in content:
    # Reading the data
    file = h5py.File(dir+i)
    images.append(np.array(file['.']['image_vector'][:]))
    ans.append(np.array(file['.']['answers'][()]))
    ques.append(np.array(file['.']['ques'][:]))
    count+=1
    if count%100 == 0:
      print("The count is:",count,"and the percentage proportion is:",round(100*count/length,2),'%')
images = tf.convert_to_tensor(np.array(images))   # For the GPU purpose
ans = tf.convert_to_tensor(np.array(ans))
ques = tf.convert_to_tensor(np.array(ques))

In [ ]:
'''  Preprocessing with the VGG 16 Model  '''

model = tf.keras.applications.VGG16(include_top=False,weights='imagenet',
        input_shape=(448,448,3))
#print("The Last layer")
last_layer = model.layers[-1].output   # Last layer has an output layer of (14,14,512)
model = Model(model.input,last_layer)
model.trainable = False


dimen_red = tf.keras.Sequential()  # Use for converting (196,512) -> (21,300)
dimen_red.add(tf.keras.layers.Conv2D(300,kernel_size=(1,1),input_shape= (14,14,512)))
dimen_red.add(tf.keras.layers.Reshape((196,300)))
dimen_red.add(tf.keras.layers.Permute((2,1)))  # Reshaping about the axis, useful for applying the dense network
dimen_red.add(tf.keras.layers.Dense(21))
dimen_red.add(tf.keras.layers.Permute((2,1)))  # Reshaping about the axis, useful for applying the dense network

In [ ]:
l = []
length = images.shape[0]
for i,j in enumerate(images):
  l.append(model(tf.reshape(j,[1,448,448,3])))  # It was not possible directly on GPU, hence had to use for loop
  if i%100 ==0:
    print("The count is:",i,"and the percentage proportion is:",round(100*i/length,2),'%')
images = tf.convert_to_tensor(np.array(l) )

In [ ]:
l = []
length = images.shape[0]
for i,j in enumerate(images):
  l.append(dimen_red(j))     # Making it to the same shape as that of question embedding
  if i%100 ==0:
    print("The count is:",i,"and the percentage proportion is:",round(100*i/length,2),'%')
images = tf.convert_to_tensor(np.array(l))


In [ ]:
images = tf.reshape(images,[length,21,300])
img = images  #Tensor containing images
que = ques  # Tensor containing question vector
img = img/255.0 # Normalizing

In [ ]:
que.shape,img.shape

In [ ]:
split_ratio = 0.8                                   # Defining the split ratio (e.g., 80% training, 20% testing)

split_index_que = int(que.shape[0] * split_ratio)    # Splitting the que tensor into training and testing
split_index_img = int(img.shape[0] * split_ratio)    # Splitting the img tensor into training and testing
que_train, que_test = tf.split(que, [split_index_que, que.shape[0] - split_index_que], axis=0)
img_train, img_test = tf.split(img, [split_index_img, img.shape[0] - split_index_img], axis=0)

print("Training question and image shape:", que_train.shape,img_train.shape)
print("Testing question and image shape:", que_test.shape,img_test.shape)


In [ ]:
''' The below mentioned two Input objects of keras will be useful for making the model '''

ques = tf.keras.layers.Input((21,300))   # Input Model (for ques)
images = tf.keras.layers.Input((21,300)) # Input Model (for images)

In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D,Flatten,Concatenate

''' Imagica is for the preprocessing of the image part'''
imagica = Dense(512,activation='tanh')(images)
imagica = Dense(512,activation='tanh')(images)


''' quesa is for the ques layer, which means preprocessing of the question layer'''
quesa = LSTM(512, dropout = 0.3,return_sequences = True,input_shape = (21,300))(ques)
quesa = Dense(512, activation = 'relu')(quesa)
quesa = Dropout(0.3)(quesa)
quesa = Dense(512, activation = 'relu')(quesa)
quesa = Dropout(0.3)(quesa)

''' Concatenating both image and the question layer'''
quesa = Concatenate()([quesa,imagica])
quesa = Flatten()(quesa)
out = tf.keras.layers.Dense(517,activation='softmax')(quesa) # Final output has 517 different categories

In [ ]:
from tensorflow.keras.utils import plot_model
model = Model([ques,images],[out])
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01),loss ='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
answers = tf.keras.utils.to_categorical(ans)
answers.shape

In [ ]:
answer = tf.keras.utils.to_categorical(ans)

split_ratio = 0.8                                      # Defining the split ratio (e.g., 80% training, 20% testing)

split_index_ans = int(answer.shape[0] * split_ratio)   # Splitting the ans tensor into training and testing
answer_train, answer_test = tf.split(answer, [split_index_ans, answer.shape[0] - split_index_ans], axis=0)

print("Answers training has shape: ", answer_train.shape)
print("Answers test has shape: ", answer_test.shape)

In [ ]:
model.summary()

In [ ]:
best_accuracy_callback = keras.callbacks.ModelCheckpoint(
    filepath='/kaggle/working/VQA_Model/best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=True,
    mode='max',
    verbose=1
)

In [ ]:
history = model.fit([img_train,que_train],answer_train,epochs = 150,batch_size=32,verbose=1,callbacks=[best_accuracy_callback])

In [ ]:
model.save('/kaggle/working/VQA_Model')

In [ ]:
prediction = tf.argmax(model.predict([img,que]),axis=1).numpy()

In [ ]:
x = open('/kaggle/input/vqa-rad-visual-question-answering-radiology/VQA_RAD Dataset Public.json','r')
train = json.load(x)
train[0]

In [ ]:
test_loss, test_accuracy = model.evaluate([img_test,que_test],answer_test)    #because of very less data we got a little less accuracy